In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import timeit
import os
from six.moves import cPickle
from text_utils import TextLoader
from tensorflow.contrib import rnn
import numpy as np
# from char_rnn_model import Model

C:\Users\ebish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Define directories, hyperparameter

In [3]:
batch_size=512
sequence_length=40
rnn_size=100
num_layers=2
lr=.001

# Load data using TextLoader object

In [4]:
text_info=TextLoader('shakespeare.txt',batch_size,sequence_length)

In [5]:
x_train=np.load('x_train_integer_data.npy')
x_val=np.load('x_val_integer_data.npy')


In [6]:
np.shape(x_train)[0]

4115968

# Create your model object

In [7]:
tf.reset_default_graph()
X = tf.placeholder(tf.int32,[None,sequence_length],name='X')
Y = tf.placeholder(tf.int32,[None,sequence_length],name='Y')
#initial_state=cell.zero_state(batch_size,tf.float32)



In [8]:
# def GRU(x):
#     GRU_cell=tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.GRUCell(num_units=n_neurons,activation=tf.nn.relu),output_size=n_outputs)
#     outputs,states=tf.nn.dynamic_rnn(GRU_cell,inputs=x,dtype=tf.float32)
#     return outputs[:,-1,:]

# def multi_rnn(x):
cells=[]

for _ in range(num_layers):
    cell=tf.contrib.rnn.GRUCell(rnn_size)
    cells.append(cell)
p=tf.contrib.rnn.MultiRNNCell(cells)
stacked_rnn_cell=tf.contrib.rnn.OutputProjectionWrapper(p,output_size=68)

outputs, states=tf.nn.dynamic_rnn(stacked_rnn_cell,inputs=tf.one_hot(X,68),dtype=tf.float32)
#     return outputs


    

In [9]:
# outputs=multi_rnn(X)
# loss = tf.reduce_mean(tf.square(outputs-Y),name='loss')
# #define optimizer
# optimizer = tf.train.AdamOptimizer(learning_rate=lr)
# train_op = optimizer.minimize(loss)

#outputs=mutli_rnn(X)
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=outputs))
opt=tf.train.AdamOptimizer(learning_rate=lr)
train_op=opt.minimize(loss)

In [21]:
def mini_batch(x_data,batch_size):
#batch size is number of samples in each iteration 
    features=x_data
    data_size=np.shape(features)[0]
    its=data_size//batch_size
    i=0
    j=batch_size
    k=0
    while k != its:
        mb_features=features[i:j,:]
        #mb_labels=labels[i:j,:]
        i+=batch_size
        j+=batch_size
        k+=1
        yield mb_features

In [27]:
np.shape(x_train[0:-1])

(4115967, 40)

# Training

In [30]:
import time
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(1):
        #fetch batch
        #acc=[]
        st=time.time()
        #for j in range(1,6):
        for a in mini_batch(x_train,batch_size):
           # batch_x = np.expand_dims(a,-1)
            batch_x=a
                #run optimization
            sess.run(train_op, feed_dict={X:batch_x[0:-1][0], Y:batch_x[1:][0]}) 
            lo=sess.run(loss,feed_dict={X:batch_x, Y:batch_x[1:][0]})
            #acc.append( sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y}))
        mid=time.time()
        #print("Epoch "+str(i)+", Time:{} ".format(mid-st)+", Loss= {:.3f}".format(lo))
        
#         batch_a,batch_b=np.expand_dims(t_val_x,-1),np.expand_dims(t_val_y,-1)
#         #batch_a=load_cifar_template1.features_reshape(batch_a)
#         #val=sess.run(accuracy, feed_dict={X:batch_a, Y:batch_b})
#         #print("Validation Accuracy:",val)
    
#     print("Training finished!")
#     testing=[]
# #     for (a,b) in load_preprocessed_test_batch(128):
# #         batch_x, batch_y = a, b
#     batch_a=np.expand_dims(t_test_x,-1)
#     testing.append(sess.run(outputs, feed_dict={X:batch_a}))
#     #print("Testing Accuracy: {:.3f}".format(np.average(testing)))

ValueError: Cannot feed value of shape (40,) for Tensor 'X:0', which has shape '(?, 40)'

# Generating Text